In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from skimage import io
from torchvision import transforms as transforms
import torchvision.transforms.functional as tf

from window.utils.tiling import create_tile_list
from window.models.yolo_for_inference import YoloClass
from window.utils.truths import windows_truth
from window.utils.inference_windows import process_annotation_df_negative_inference, create_windows_from_yolo, windows_to_whole_im, xc_to_xmn

In [ ]:
valid_whole_image_dir_inria = "/data/old_home_dir/ChrissyF/INRIA/Valid/whole_images/"
truth_file_inria = "/data/old_home_dir/ChrissyF/INRIA/00INRIA_bboxes_valid_yolo.csv"

valid_whole_image_dir_gfrc = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
truth_file_gfrc = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"

valid_whole_image_dir_vedai = "/data/old_home_dir/ChrissyF/VEDAI/Valid/whole_images/"
truth_file_vedai = "/data/old_home_dir/ChrissyF/VEDAI/Valid/00VEDAI_test_bboxes.csv"

In [ ]:
output_base_dir_inria = "/home/cmf21/pytorch_save/INRIA/rgb_baseline/"
output_base_dir_vedai = "/home/cmf21/pytorch_save/VEDAI/Bin/rgb_baseline/"
output_base_dir_gfrc = "/home/cmf21/pytorch_save/GFRC/Bin/rgb_baseline2/"
output_base_dir_gfrc_hnm = "/home/cmf21/pytorch_save/GFRC/Bin/rgb_baseline2_hnm/"
output_base_dir_inria_grey = "/home/cmf21/pytorch_save/INRIA/grey_baseline/"
output_base_dir_vedai_grey = "/home/cmf21/pytorch_save/VEDAI/Bin/grey_baseline/"
output_base_dir_gfrc_grey = "/home/cmf21/pytorch_save/GFRC/Bin/grey_baseline/"
output_base_dir_vedai_multi = "/home/cmf21/pytorch_save/VEDAI/Multi/rgb_baseline/"
output_base_dir_gfrc_multi = "/home/cmf21/pytorch_save/GFRC/Multi/rgb_baseline/"
output_base_dir_vedai_multi_grey = "/home/cmf21/pytorch_save/VEDAI/Multi/grey_baseline/"
output_base_dir_gfrc_multi_grey = "/home/cmf21/pytorch_save/GFRC/Multi/grey_baseline/"

In [ ]:
experiment = 'gfrc_hnm'
predict_results = True

if experiment == 'inria_bin':
    output_base_dir = output_base_dir_inria
    valid_whole_image_dir = valid_whole_image_dir_inria
    truth_file = truth_file_inria
    predicted_object_loc = output_base_dir + 'boxes_out236_full.csv'
    saveweightspath = output_base_dir + "testing_save_236.pt"
    channels_in = 3
    nclazz = 1
    dataset = 'inria'
elif experiment == 'inria_bin_grey':
    output_base_dir = output_base_dir_inria_grey
    valid_whole_image_dir = valid_whole_image_dir_inria
    truth_file = truth_file_inria
    predicted_object_loc = output_base_dir + 'boxes_out63_full.csv'
    saveweightspath = output_base_dir + "testing_save_63.pt"
    channels_in = 1
    nclazz = 1
    dataset = 'inria'
elif experiment == 'vedai_bin':
    output_base_dir = output_base_dir_vedai
    valid_whole_image_dir = valid_whole_image_dir_vedai
    truth_file = truth_file_vedai
    predicted_object_loc = output_base_dir + 'boxes_out922_full.csv'
    saveweightspath = output_base_dir + "testing_save_922.pt"
    channels_in = 3
    nclazz = 1
    dataset = 'vedai'
elif experiment == 'vedai_bin_grey':
    output_base_dir = output_base_dir_vedai_grey
    valid_whole_image_dir = valid_whole_image_dir_vedai
    truth_file = truth_file_vedai
    predicted_object_loc = output_base_dir + 'boxes_out484_full.csv'
    saveweightspath = output_base_dir + "testing_save_484.pt"
    channels_in = 1
    nclazz = 1
    dataset = 'vedai'
elif experiment == 'vedai_multi':
    output_base_dir = output_base_dir_vedai_multi
    valid_whole_image_dir = valid_whole_image_dir_vedai
    truth_file = truth_file_vedai
    predicted_object_loc = output_base_dir + 'boxes_out111_full.csv'
    saveweightspath = output_base_dir + "testing_save_111.pt"
    channels_in = 3
    nclazz = 9
    dataset = 'vedai'
elif experiment == 'vedai_multi_grey':
    output_base_dir = output_base_dir_vedai_multi_grey
    valid_whole_image_dir = valid_whole_image_dir_vedai
    truth_file = truth_file_vedai
    predicted_object_loc = output_base_dir + 'boxes_out452_full.csv'
    saveweightspath = output_base_dir + "testing_save_452.pt"
    channels_in = 1
    nclazz = 9
    dataset = 'vedai'
elif experiment == 'gfrc_bin':
    output_base_dir = output_base_dir_gfrc
    valid_whole_image_dir = valid_whole_image_dir_gfrc
    truth_file = truth_file_gfrc
    predicted_object_loc = output_base_dir + 'boxes_out163_full.csv'
    saveweightspath = output_base_dir + "testing_save_163.pt"
    channels_in = 3
    nclazz = 1
    dataset = 'gfrc'
elif experiment == 'gfrc_bin_grey':
    output_base_dir = output_base_dir_gfrc_grey
    valid_whole_image_dir = valid_whole_image_dir_gfrc
    truth_file = truth_file_gfrc
    predicted_object_loc = output_base_dir + 'boxes_out265_full.csv'
    saveweightspath = output_base_dir + "testing_save_265.pt"
    channels_in = 1
    nclazz = 1
    dataset = 'gfrc'
elif experiment == 'gfrc_multi':
    output_base_dir = output_base_dir_gfrc_multi
    valid_whole_image_dir = valid_whole_image_dir_gfrc
    truth_file = truth_file_gfrc
    predicted_object_loc = output_base_dir + 'boxes_out189_full.csv'
    saveweightspath = output_base_dir + "testing_save_189.pt"
    channels_in = 3
    nclazz = 6
    dataset = 'gfrc'
elif experiment == 'gfrc_multi_grey':
    output_base_dir = output_base_dir_gfrc_multi_grey
    valid_whole_image_dir = valid_whole_image_dir_gfrc
    truth_file = truth_file_gfrc
    predicted_object_loc = output_base_dir + 'boxes_out274_full.csv'
    saveweightspath = output_base_dir + "testing_save_274.pt"
    channels_in = 1
    nclazz = 6
    dataset = 'gfrc'
elif experiment == 'gfrc_hnm':
    output_base_dir = output_base_dir_gfrc_hnm
    valid_whole_image_dir = valid_whole_image_dir_gfrc
    truth_file = truth_file_gfrc
    predicted_object_loc = output_base_dir + 'boxes_out97_full.csv'
    saveweightspath = output_base_dir + "testing_save_97.pt"
    channels_in = 3
    nclazz = 2
    dataset = 'gfrc'

In [ ]:
output_dir = output_base_dir + "valid_images_out/"
output_csv = output_dir + "valid_results/results_matched_nms.csv"

In [ ]:
# get all image files
image_files_jpg = list(Path(valid_whole_image_dir).glob("*.jpg"))
image_files_png = list(Path(valid_whole_image_dir).glob("*.png"))
image_files = image_files_jpg + image_files_png
image_files = [img.name for img in image_files]

In [ ]:
# Process truth files
truths = pd.read_csv(truth_file)
if dataset == 'gfrc':
    truths.loc[:, 'filename'] = [strin.replace('/', '_') for strin in truths.file_loc]
    truths = windows_truth(truths)
else:
    truths['filename'] = truths['file_loc']
    truths['xmn'] = truths['xmin']
    truths['xmx'] = truths['xmax']
    truths['ymn'] = truths['ymin']
    truths['ymx'] = truths['ymax']
if dataset == 'vedai':
    truths['oc'] = truths['class_']

In [ ]:
def split_filename(str_in):
    file_nm = str_in[:-4]
    file_splt = file_nm.split('_')
    file_out = file_splt[0] + '_' + file_splt[1] + '.jpg'
    tile_out = file_splt[2]
    return file_out, tile_out

In [ ]:
# create yolo model


In [ ]:
if predict_results:
    if dataset == 'inria':
        yolo_model = YoloClass(wtpath=saveweightspath, channels=channels_in, img_w=640, img_h=480, nclazz=nclazz)
    elif dataset == 'vedai':
        yolo_model = YoloClass(wtpath=saveweightspath, channels=channels_in, img_w=1024, img_h=1024, nclazz=nclazz)
    else:
        yolo_model = YoloClass(wtpath=saveweightspath, channels=channels_in, nclazz=nclazz)
        
    windows_whole = pd.DataFrame(columns=['xc', 'yc', 'wid', 'hei', 'conf', 'class', 'xmn', 'xmx', 'ymn', 'ymx', 'filename'])

    for fl in image_files:
        # Per image
        
        if channels_in == 3:
            #whole_im = cv2.imread(valid_whole_image_dir + fl)
            #whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)
            whole_im = io.imread(valid_whole_image_dir + fl, as_gray=False)
        else:
            #whole_im = cv2.imread(valid_whole_image_dir + fl, cv2.IMREAD_GRAYSCALE)
            #whole_im = np.array(whole_im / 255, dtype=np.float32)
            whole_im = io.imread(valid_whole_image_dir + fl, as_gray=True)
            whole_im = np.array(np.divide(whole_im, 255.0), dtype=np.float32)

        orig_im_size = whole_im.shape
        
        if dataset == 'inria':
            whole_im = cv2.resize(whole_im, (640, 480))

        # create tiles
        if dataset == 'gfrc':
            tilez = create_tile_list(whole_im)
        else:
            tilez = [whole_im]

        # get predictions from yolo
        boxes_whole_im = yolo_model.inference_on_image(tilez, 0.005)

        if dataset == 'gfrc':
            windows_whole_im = windows_to_whole_im(boxes_whole_im)
        else:
            windows_whole_im = xc_to_xmn(boxes_whole_im, orig_im_size[1], orig_im_size[0])
        
        windows_whole_im['filename'] = fl

        windows_whole = pd.concat((windows_whole, windows_whole_im), axis=0)


In [ ]:
windows_whole

In [ ]:
if not predict_results:
    # read in results
    predicted_objects = pd.read_csv(predicted_object_loc)
    file_locs = predicted_objects.file
    file_locs = [Path(fl).name for fl in file_locs]
    if dataset == 'gfrc':
        filenames = [split_filename(fl)[0] for fl in file_locs]
        tiles = [split_filename(fl)[1] for fl in file_locs]
        predicted_objects['filename'] = filenames
        predicted_objects['tile'] = tiles
        windows_whole = windows_to_whole_im(predicted_objects)
    elif dataset == 'inria':
        predicted_objects['filename'] = file_locs
        windows_whole = pd.DataFrame(columns=['xc', 'yc', 'wid', 'hei', 'file', 'conf', 'class', 'tp', 'filename', 'xmn', 'xmx', 'ymn', 'ymx'])
        for idx, fl in enumerate(image_files):
            predicted_obs_im = predicted_objects[predicted_objects.filename == fl]
            img_in = cv2.imread(valid_whole_image_dir + fl)
            windows_im = xc_to_xmn(predicted_obs_im, img_in.shape[1], img_in.shape[0])
            windows_whole = pd.concat((windows_whole, windows_im), axis=0)
    elif dataset == 'vedai':
        predicted_objects['filename'] = file_locs
        windows_whole = xc_to_xmn(predicted_objects, 1024, 1024)


In [ ]:
np.max(windows_whole.conf)

In [ ]:
windows_filename = experiment + '_windows.csv'
output_dir = "/home/cmf21/pytorch_save/output_for_draft/"
windows_whole.to_csv(output_dir + windows_filename, index=False)

In [ ]:
fn_df = results_all_ims[results_all_ims.confmat == 'FN']
# [0, 1856, 3712, 5504] [1856, 3712, 5568, 7360] [0, 1248, 2496, 3664] [1248, 2496, 3744, 4912]
def is_tile_edge(row_in):
    xmn = row_in.xmn - 10
    xmx = row_in.xmx + 10
    ymn = row_in.ymn - 10
    ymx = row_in.ymx + 10
    
    x1 = xmn < 0 < xmx
    x2 = xmn < 1856 < xmx
    x3 = xmn < 3712 < xmx
    x4 = xmn < 5504 < xmx
    x5 = xmn < 5568 < xmx
    x6 = xmn < 7360 < xmx
    y1 = ymn < 0 < ymx
    y2 = ymn < 1248 < ymx
    y3 = ymn < 2496 < ymx
    y4 = ymn < 3664 < ymx
    y5 = ymn < 3744 < ymx
    y6 = ymn < 4912 < ymx
    edge = x1 | x2 | x3 | x4 | x5 | x6 | y1 | y2 | y3 | y4 | y5 | y6
    
    return edge

fn_edge = []
for nn in range(fn_df.shape[0]):
    fn_edge.append(is_tile_edge(fn_df.iloc[nn, :]))
    
print(np.sum(fn_edge)/fn_df.shape[0])

# calculate percentage of image that is close to edge of tile
(600 * 4912 + 600 * 7360 - 600 * 600) / (4912 * 7360)